# Demo - Performance Comparison (CIFAR10)

## 1. Load CIAFR10

In [1]:
# https://github.com/RobustBench/robustbench
from robustbench.data import load_cifar10
from robustbench.utils import load_model, clean_accuracy
from utils import l2_distance

images, labels = load_cifar10(n_examples=50)
device = "cuda"

Files already downloaded and verified


## 2. Standard Accuracy

In [2]:
model_list = ['Standard', 'Wong2020Fast', 'Rice2020Overfitting']
for model_name in model_list:
    model = load_model(model_name, norm='Linf').to(device)
    acc = clean_accuracy(model, images.to(device), labels.to(device))
    print('Model: {}'.format(model_name))
    print('- Standard Acc: {}'.format(acc))

Model: Standard
- Standard Acc: 0.96
Model: Wong2020Fast
- Standard Acc: 0.92
Model: Rice2020Overfitting
- Standard Acc: 0.86


## 3. Torchattacks, Foolbox and ART

In [3]:
import datetime
import numpy as np
import warnings

warnings.filterwarnings(action='ignore')

import torch
import torch.nn as nn
import torch.optim as optim

# https://github.com/Harry24k/adversarial-attacks-pytorch
import torchattacks
print("torchattacks %s"%(torchattacks.__version__))

# https://github.com/bethgelab/foolbox
import foolbox as fb
print("foolbox %s"%(fb.__version__))

# https://github.com/IBM/adversarial-robustness-toolbox
import art
import art.attacks.evasion as evasion
from art.classifiers import PyTorchClassifier
print("art %s"%(art.__version__))

torchattacks 3.0.0
foolbox 3.3.1
art 1.7.0


<ipython-input-3-36788938c0b3>:22: DeprecationWarning: The module art.classifiers will be removed in ART 1.8.0 and replaced with art.estimators.classification
  from art.classifiers import PyTorchClassifier


## 3.1. Linf

### FGSM

In [4]:
for model_name in model_list:
    print('Model: {}'.format(model_name))
    model = load_model(model_name, norm='Linf').to(device)
    
    print("- Torchattacks")
    atk = torchattacks.FGSM(model, eps=8/255)
    start = datetime.datetime.now()
    adv_images = atk(images, labels)
    end = datetime.datetime.now()
    acc = clean_accuracy(model, adv_images, labels)
    print('- Robust Acc: {} ({} ms)'.format(acc, int((end-start).total_seconds()*1000)))
    
    print("- Foolbox")
    fmodel = fb.PyTorchModel(model, bounds=(0, 1))
    atk = fb.attacks.LinfFastGradientAttack(random_start=False)
    start = datetime.datetime.now()
    _, adv_images, _ = atk(fmodel, images.to('cuda:0'), labels.to('cuda:0'), epsilons=8/255)
    end = datetime.datetime.now()
    acc = clean_accuracy(model, adv_images, labels)
    print('- Robust Acc: {} ({} ms)'.format(acc, int((end-start).total_seconds()*1000)))
    
    print("- ART")
    classifier = PyTorchClassifier(model=model, clip_values=(0, 1),
                                   loss=nn.CrossEntropyLoss(),
                                   optimizer=optim.Adam(model.parameters(), lr=0.01),
                                   input_shape=(3, 32, 32), nb_classes=10)
    atk = evasion.FastGradientMethod(norm=np.inf, batch_size=50,
                                     estimator=classifier, eps=8/255)
    start = datetime.datetime.now()
    adv_images = torch.tensor(atk.generate(images, labels)).to(device)
    end = datetime.datetime.now()
    acc = clean_accuracy(model, adv_images, labels)
    print('- Robust Acc: {} ({} ms)'.format(acc, int((end-start).total_seconds()*1000)))
    
    print()

C:\Users\slcf\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Model: Standard
- Torchattacks
- Robust Acc: 0.34 (54 ms)
- Foolbox
- Robust Acc: 0.34 (15 ms)
- ART
- Robust Acc: 0.34 (214 ms)

Model: Wong2020Fast
- Torchattacks
- Robust Acc: 0.48 (5 ms)
- Foolbox
- Robust Acc: 0.48 (8 ms)
- ART
- Robust Acc: 0.48 (59 ms)

Model: Rice2020Overfitting
- Torchattacks
- Robust Acc: 0.62 (82 ms)
- Foolbox
- Robust Acc: 0.62 (30 ms)
- ART
- Robust Acc: 0.62 (768 ms)



### BIM

In [5]:
for model_name in model_list:
    print('Model: {}'.format(model_name))
    model = load_model(model_name, norm='Linf').to(device)
    
    print("- Torchattacks")
    atk = torchattacks.BIM(model, eps=8/255, alpha=2/255, steps=10)
    start = datetime.datetime.now()
    adv_images = atk(images, labels)
    end = datetime.datetime.now()
    acc = clean_accuracy(model, adv_images, labels)
    print('- Robust Acc: {} ({} ms)'.format(acc, int((end-start).total_seconds()*1000)))
    
    print("- Foolbox")
    fmodel = fb.PyTorchModel(model, bounds=(0, 1))
    atk = fb.attacks.LinfBasicIterativeAttack(abs_stepsize=2/255, steps=10, random_start=False)
    start = datetime.datetime.now()
    _, adv_images, _ = atk(fmodel, images.to('cuda:0'), labels.to('cuda:0'), epsilons=8/255)
    end = datetime.datetime.now()
    acc = clean_accuracy(model, adv_images, labels)
    print('- Robust Acc: {} ({} ms)'.format(acc, int((end-start).total_seconds()*1000)))
    
    print("- ART")
    classifier = PyTorchClassifier(model=model, clip_values=(0, 1),
                                   loss=nn.CrossEntropyLoss(),
                                   optimizer=optim.Adam(model.parameters(), lr=0.01),
                                   input_shape=(3, 32, 32), nb_classes=10)
    atk = evasion.BasicIterativeMethod(batch_size=50,
                                       estimator=classifier, eps=8/255,
                                       eps_step=2/255, max_iter=10)
    start = datetime.datetime.now()
    adv_images = torch.tensor(atk.generate(images, labels)).to(device)
    end = datetime.datetime.now()
    acc = clean_accuracy(model, adv_images, labels)
    print('- Robust Acc: {} ({} ms)'.format(acc, int((end-start).total_seconds()*1000)))
    
    print()

Model: Standard
- Torchattacks
- Robust Acc: 0.0 (196 ms)
- Foolbox
- Robust Acc: 0.0 (366 ms)
- ART


PGD - Batches:   0%|          | 0/1 [00:00<?, ?it/s]

- Robust Acc: 0.0 (1360 ms)

Model: Wong2020Fast
- Torchattacks
- Robust Acc: 0.44 (49 ms)
- Foolbox
- Robust Acc: 0.44 (53 ms)
- ART


PGD - Batches:   0%|          | 0/1 [00:00<?, ?it/s]

- Robust Acc: 0.44 (420 ms)

Model: Rice2020Overfitting
- Torchattacks
- Robust Acc: 0.58 (1369 ms)
- Foolbox
- Robust Acc: 0.58 (1861 ms)
- ART


PGD - Batches:   0%|          | 0/1 [00:00<?, ?it/s]

- Robust Acc: 0.58 (4740 ms)



### PGD

In [6]:
for model_name in model_list:
    print('Model: {}'.format(model_name))
    model = load_model(model_name, norm='Linf').to(device)
    
    print("- Torchattacks")
    atk = torchattacks.PGD(model, eps=8/255, alpha=2/255, steps=10, random_start=False)
    start = datetime.datetime.now()
    adv_images = atk(images, labels)
    end = datetime.datetime.now()
    acc = clean_accuracy(model, adv_images, labels)
    print('- Robust Acc: {} ({} ms)'.format(acc, int((end-start).total_seconds()*1000)))
    
    print("- Foolbox")
    fmodel = fb.PyTorchModel(model, bounds=(0, 1))
    atk = fb.attacks.LinfPGD(abs_stepsize=2/255, steps=10, random_start=False)
    start = datetime.datetime.now()
    _, adv_images, _ = atk(fmodel, images.to('cuda:0'), labels.to('cuda:0'), epsilons=8/255)
    end = datetime.datetime.now()
    acc = clean_accuracy(model, adv_images, labels)
    print('- Robust Acc: {} ({} ms)'.format(acc, int((end-start).total_seconds()*1000)))
    
    print("- ART")
    classifier = PyTorchClassifier(model=model, clip_values=(0, 1),
                                   loss=nn.CrossEntropyLoss(),
                                   optimizer=optim.Adam(model.parameters(), lr=0.01),
                                   input_shape=(3, 32, 32), nb_classes=10)
    atk = evasion.ProjectedGradientDescent(batch_size=50, num_random_init=0,
                                           estimator=classifier, eps=8/255,
                                           eps_step=2/255, max_iter=10)
    start = datetime.datetime.now()
    adv_images = torch.tensor(atk.generate(images, labels)).to(device)
    end = datetime.datetime.now()
    acc = clean_accuracy(model, adv_images, labels)
    print('- Robust Acc: {} ({} ms)'.format(acc, int((end-start).total_seconds()*1000)))
    
    print()

Model: Standard
- Torchattacks
- Robust Acc: 0.0 (174 ms)
- Foolbox
- Robust Acc: 0.0 (354 ms)
- ART


PGD - Batches:   0%|          | 0/1 [00:00<?, ?it/s]

- Robust Acc: 0.0 (1384 ms)

Model: Wong2020Fast
- Torchattacks
- Robust Acc: 0.44 (52 ms)
- Foolbox
- Robust Acc: 0.44 (56 ms)
- ART


PGD - Batches:   0%|          | 0/1 [00:00<?, ?it/s]

- Robust Acc: 0.44 (437 ms)

Model: Rice2020Overfitting
- Torchattacks
- Robust Acc: 0.58 (1348 ms)
- Foolbox
- Robust Acc: 0.58 (1856 ms)
- ART


PGD - Batches:   0%|          | 0/1 [00:00<?, ?it/s]

- Robust Acc: 0.58 (4704 ms)



## 3.2. L2

## DeepFool

In [7]:
for model_name in model_list:
    print('Model: {}'.format(model_name))
    model = load_model(model_name, norm='Linf').to(device)
    
    print("- Torchattacks")
    atk = torchattacks.DeepFool(model, steps=50, overshoot=0.02)
    start = datetime.datetime.now()
    adv_images = atk(images, labels)
    end = datetime.datetime.now()
    acc = clean_accuracy(model, adv_images, labels)
    l2 = l2_distance(model, images, adv_images, labels, device=device)
    print('- Robust Acc: {} / L2: {:1.2} ({} ms)'.format(acc, l2,
                                                         int((end-start).total_seconds()*1000)))
    
    print("- Foolbox")
    fmodel = fb.PyTorchModel(model, bounds=(0, 1))
    atk = fb.attacks.L2DeepFoolAttack(steps=50, candidates=10, overshoot=0.02)
    start = datetime.datetime.now()
    _, adv_images, _ = atk(fmodel, images.to('cuda'), labels.to('cuda'), epsilons=1)
    end = datetime.datetime.now()
    acc = clean_accuracy(model, adv_images, labels)
    l2 = l2_distance(model, images, adv_images, labels, device=device)
    print('- Robust Acc: {} / L2: {:1.2} ({} ms)'.format(acc, l2,
                                                         int((end-start).total_seconds()*1000)))

    print("- ART")
    classifier = PyTorchClassifier(model=model, clip_values=(0, 1),
                                   loss=nn.CrossEntropyLoss(),
                                   optimizer=optim.Adam(model.parameters(), lr=0.01),
                                   input_shape=(3, 32, 32), nb_classes=10)
    atk = evasion.DeepFool(classifier=classifier, max_iter=50,
                           batch_size=50)

    start = datetime.datetime.now()
    adv_images = torch.tensor(atk.generate(images, labels)).to(device)
    end = datetime.datetime.now()
    acc = clean_accuracy(model, adv_images, labels)
    l2 = l2_distance(model, images, adv_images, labels, device=device)
    print('- Robust Acc: {} / L2: {:1.2} ({} ms)'.format(acc, l2,
                                                         int((end-start).total_seconds()*1000)))
    
    print()

Model: Standard
- Torchattacks
- Robust Acc: 0.02 / L2: 0.25 (17210 ms)
- Foolbox
- Robust Acc: 0.02 / L2: 0.23 (6837 ms)
- ART


DeepFool:   0%|          | 0/1 [00:00<?, ?it/s]

- Robust Acc: 0.14 / L2: 0.25 (46350 ms)

Model: Wong2020Fast
- Torchattacks
- Robust Acc: 0.0 / L2: 1.0 (3847 ms)
- Foolbox
- Robust Acc: 0.4 / L2: 0.41 (1786 ms)
- ART


DeepFool:   0%|          | 0/1 [00:00<?, ?it/s]

- Robust Acc: 0.08 / L2: 1.1 (16766 ms)

Model: Rice2020Overfitting
- Torchattacks
- Robust Acc: 0.0 / L2: 1.2 (33602 ms)
- Foolbox
- Robust Acc: 0.36 / L2: 0.47 (20423 ms)
- ART


DeepFool:   0%|          | 0/1 [00:00<?, ?it/s]

- Robust Acc: 0.1 / L2: 1.3 (165491 ms)



### CW

In [8]:
for model_name in model_list:
    print('Model: {}'.format(model_name))
    model = load_model(model_name, norm='Linf').to(device)
    
    print("- Torchattacks")
    atk = torchattacks.CW(model, c=1, kappa=0, steps=100, lr=0.01)
    start = datetime.datetime.now()
    adv_images = atk(images, labels)
    end = datetime.datetime.now()
    acc = clean_accuracy(model, adv_images, labels)
    l2 = l2_distance(model, images, adv_images, labels, device=device)
    print('- Robust Acc: {} / L2: {:1.2} ({} ms)'.format(acc, l2,
                                                         int((end-start).total_seconds()*1000)))
    
    print("- Foolbox")
    fmodel = fb.PyTorchModel(model, bounds=(0, 1))
    atk = fb.attacks.L2CarliniWagnerAttack(binary_search_steps=1, initial_const=1,
                                           confidence=0, steps=100, stepsize=0.01)
    start = datetime.datetime.now()
    _, adv_images, _ = atk(fmodel, images.to('cuda'), labels.to('cuda'), epsilons=1)
    end = datetime.datetime.now()
    acc = clean_accuracy(model, adv_images, labels)
    l2 = l2_distance(model, images, adv_images, labels, device=device)
    print('- Robust Acc: {} / L2: {:1.2} ({} ms)'.format(acc, l2,
                                                         int((end-start).total_seconds()*1000)))

    print("- ART")
    classifier = PyTorchClassifier(model=model, clip_values=(0, 1),
                                   loss=nn.CrossEntropyLoss(),
                                   optimizer=optim.Adam(model.parameters(), lr=0.01),
                                   input_shape=(3, 32, 32), nb_classes=10)
    atk = evasion.CarliniL2Method(batch_size=50, classifier=classifier, 
                                  binary_search_steps=1, initial_const=1,
                                  confidence=0, max_iter=100,
                                  learning_rate=0.01)
    start = datetime.datetime.now()
    adv_images = torch.tensor(atk.generate(images, labels)).to(device)
    end = datetime.datetime.now()
    acc = clean_accuracy(model, adv_images, labels)
    l2 = l2_distance(model, images, adv_images, labels, device=device)
    print('- Robust Acc: {} / L2: {:1.2} ({} ms)'.format(acc, l2,
                                                         int((end-start).total_seconds()*1000)))
    
    print()

Model: Standard
- Torchattacks
- Robust Acc: 0.0 / L2: 0.4 (2596 ms)
- Foolbox
- Robust Acc: 0.0 / L2: 0.4 (2668 ms)
- ART


C&W L_2:   0%|          | 0/1 [00:00<?, ?it/s]

- Robust Acc: 0.0 / L2: 0.59 (196738 ms)

Model: Wong2020Fast
- Torchattacks
- Robust Acc: 0.14 / L2: 0.61 (3795 ms)
- Foolbox
- Robust Acc: 0.32 / L2: 0.41 (3928 ms)
- ART


C&W L_2:   0%|          | 0/1 [00:00<?, ?it/s]

- Robust Acc: 0.22 / L2: 0.7 (66067 ms)

Model: Rice2020Overfitting
- Torchattacks
- Robust Acc: 0.22 / L2: 0.56 (43484 ms)
- Foolbox
- Robust Acc: 0.34 / L2: 0.43 (44418 ms)
- ART


C&W L_2:   0%|          | 0/1 [00:00<?, ?it/s]

- Robust Acc: 0.26 / L2: 0.65 (694972 ms)



### PGD L2

In [9]:
for model_name in model_list:
    print('Model: {}'.format(model_name))
    model = load_model(model_name, norm='Linf').cuda()
    
    print("- Torchattacks")
    atk = torchattacks.PGDL2(model, eps=128/255, alpha=15/255, steps=10, random_start=False)
    start = datetime.datetime.now()
    adv_images = atk(images, labels)
    end = datetime.datetime.now()
    acc = clean_accuracy(model, adv_images, labels)
    l2 = l2_distance(model, images, adv_images, labels, device=device)
    print('- Robust Acc: {} / L2: {:1.2} ({} ms)'.format(acc, l2,
                                                         int((end-start).total_seconds()*1000)))
    
    print("- Foolbox")
    fmodel = fb.PyTorchModel(model, bounds=(0, 1))
    atk = fb.attacks.L2PGD(abs_stepsize=15/255, steps=10, random_start=False)
    start = datetime.datetime.now()
    _, adv_images, _ = atk(fmodel, images.to('cuda:0'), labels.to('cuda:0'), epsilons=128/255)
    end = datetime.datetime.now()
    acc = clean_accuracy(model, adv_images, labels)
    l2 = l2_distance(model, images, adv_images, labels, device=device)
    print('- Robust Acc: {} / L2: {:1.2} ({} ms)'.format(acc, l2,
                                                         int((end-start).total_seconds()*1000)))
    
    print("- ART")
    classifier = PyTorchClassifier(model=model, clip_values=(0, 1),
                                   loss=nn.CrossEntropyLoss(),
                                   optimizer=optim.Adam(model.parameters(), lr=0.01),
                                   input_shape=(3, 32, 32), nb_classes=10)
    atk = evasion.ProjectedGradientDescent(batch_size=50, num_random_init=0,
                                           norm = 2, estimator=classifier, eps=128/255,
                                           eps_step=15/255, max_iter=10)
    start = datetime.datetime.now()
    adv_images = torch.tensor(atk.generate(images, labels)).to(device)
    end = datetime.datetime.now()
    acc = clean_accuracy(model, adv_images, labels)
    l2 = l2_distance(model, images, adv_images, labels, device=device)
    print('- Robust Acc: {} / L2: {:1.2} ({} ms)'.format(acc, l2,
                                                         int((end-start).total_seconds()*1000)))
    
    print()

Model: Standard
- Torchattacks
- Robust Acc: 0.06 / L2: 0.41 (184 ms)
- Foolbox
- Robust Acc: 0.06 / L2: 0.41 (396 ms)
- ART


PGD - Batches:   0%|          | 0/1 [00:00<?, ?it/s]

- Robust Acc: 0.06 / L2: 0.4 (1364 ms)

Model: Wong2020Fast
- Torchattacks
- Robust Acc: 0.68 / L2: 0.5 (52 ms)
- Foolbox
- Robust Acc: 0.68 / L2: 0.5 (57 ms)
- ART


PGD - Batches:   0%|          | 0/1 [00:00<?, ?it/s]

- Robust Acc: 0.68 / L2: 0.5 (429 ms)

Model: Rice2020Overfitting
- Torchattacks
- Robust Acc: 0.7 / L2: 0.5 (1377 ms)
- Foolbox
- Robust Acc: 0.7 / L2: 0.5 (1968 ms)
- ART


PGD - Batches:   0%|          | 0/1 [00:00<?, ?it/s]

- Robust Acc: 0.7 / L2: 0.5 (4777 ms)

